# Academy of Py - Setup

In [1]:
import pandas as pd
import numpy as np

# File to Load 
schools = "Resources/schools_complete.csv"
students = "Resources/students_complete.csv"

pd.options.display.float_format = '{:,.3f}'.format

# Read-in csvs:
students_df = pd.read_csv(students)
schools_df = pd.read_csv(schools)

# Totals for district:
school_list = students_df['school_name'].unique()
total_schools = len(school_list)
total_students = students_df['student_name'].count()
total_budget = schools_df['budget'].sum()

# Averages for scores:
ave_math_score=students_df['math_score'].mean()
ave_reading_score=students_df['reading_score'].mean()

# Locate and count passing
math_pass_df = students_df.loc[students_df["math_score"] >= 70, :]
math_pass_count = math_pass_df['math_score'].count()
reading_pass_df = students_df.loc[students_df["reading_score"] >= 70, :]
reading_pass_count = reading_pass_df['reading_score'].count()

# Calculate percentages:
percent_pass_math = math_pass_count/total_students*100
percent_pass_reading = reading_pass_count/total_students*100
percent_pass_overall = (math_pass_count + reading_pass_count)/total_students*50

# Build district dataframe:
district_breakdown = pd.DataFrame({"Total Schools": [total_schools],
                                   "Total Students": [total_students],
                                   "Total Budget": [total_budget],
                                   "Average Math Score": [ave_math_score],
                                   "Average Reading Score": [ave_reading_score],
                                   "% Passing Math":[percent_pass_math],
                                   "% Passing Reading":[percent_pass_reading],
                                   "% Overall Passing Rate": [percent_pass_overall]})
district_breakdown['Total Budget'] = district_breakdown['Total Budget'].map('${:,.2f}'.format)
district_breakdown=district_breakdown[['Total Schools','Total Students','Total Budget','Average Math Score', 'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]

district_breakdown

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985,81.878,74.981,85.805,80.393


# Overview table that summarizes key metrics about each school

In [2]:
# Schools df for merge with average students scores:

schools_df = schools_df.rename(index=str, columns={"student_name":"school_name"})
school_groups_df =students_df.groupby(["school_name"])
school_score_groups_df = school_groups_df[['school_name','math_score','reading_score']].mean().reset_index()

# Merge schools data with average scores data:
aves_merge_df = pd.merge(school_score_groups_df, schools_df, on="school_name")

# Pass counts by school and merge with school summary df:
school_math_pass_group  = math_pass_df.groupby(["school_name"])
school_math_pass_df = school_math_pass_group[['math_score']].count().reset_index()
school_reading_pass_group  = reading_pass_df.groupby(["school_name"])
school_reading_pass_df = school_reading_pass_group[['reading_score']].count().reset_index()
pass_count_merge_df = pd.merge(aves_merge_df, school_math_pass_df, on="school_name")
pass_count_merge_df = pd.merge(pass_count_merge_df, school_reading_pass_df, on="school_name")

# per school summary with percentages columns and per student budgets:
pass_count_merge_df['% Passing Math'] = pass_count_merge_df['math_score_y']/pass_count_merge_df['size']*100
pass_count_merge_df['% Passing Reading'] = pass_count_merge_df['reading_score_y']/pass_count_merge_df['size']*100
pass_count_merge_df['% Overall Passing'] = (pass_count_merge_df['% Passing Math'] + pass_count_merge_df['% Passing Reading'])/2
pass_count_merge_df['Per Student Budget'] = pass_count_merge_df['budget']/pass_count_merge_df['size']

school_summary = pass_count_merge_df.rename(index=str, columns={"school_name":"School",'math_score_x':'Average Math Score',
                                                                'reading_score_x':'Average Reading Score','type':'School Type',
                                                                'size':'Total Students','budget':'Total Budget'})

school_summary = school_summary.drop(['math_score_y','reading_score_y','School ID'], axis = 1)
school_summary = school_summary[["School",'School Type','Total Students','Total Budget','Per Student Budget',
                                 'Average Math Score', 'Average Reading Score','% Passing Math',
                                 '% Passing Reading','% Overall Passing']]
school_summary = school_summary.set_index('School')
school_summary['Total Budget'] = school_summary['Total Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${:,.2f}'.format)

# Top Performing Schools

In [3]:
ranked_schools = school_summary.sort_values(by=['% Overall Passing'], ascending=False)
ranked_schools.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,95.587
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418,83.849,93.272,97.309,95.291
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,95.270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,95.266
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,95.204


# Bottom Performing Schools(Passing Rate)

In [4]:
ranked_schools.tail().sort_values("Total Students", ascending = False)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,73.640
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.843,80.745,66.367,80.220,73.293
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,73.364
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,73.500
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,73.804


# Math Scores by Grade

In [5]:
students_9th_df=students_df.loc[students_df['grade'] == '9th',:]
students_10th_df=students_df.loc[students_df['grade'] == '10th',:]
students_11th_df=students_df.loc[students_df['grade'] == '11th',:]
students_12th_df=students_df.loc[students_df['grade'] == '12th',:]

students_9th_df=students_9th_df.rename(index=str, columns={"school_name":"School"})
students_10th_df=students_10th_df.rename(index=str, columns={"school_name":"School"})
students_11th_df=students_11th_df.rename(index=str, columns={"school_name":"School"})
students_12th_df=students_12th_df.rename(index=str, columns={"school_name":"School"})



# Groupbys for each grade

students_9th_df = students_9th_df.set_index('School')
students_10th_df = students_10th_df.set_index('School')
students_11th_df = students_11th_df.set_index('School')
students_12th_df = students_12th_df.set_index('School')

grade_group_9th_df = students_9th_df.groupby(['School'])
math_scores_9th_df = grade_group_9th_df['math_score'].mean()

grade_group_10th_df = students_10th_df.groupby(['School'])
math_scores_10th_df = grade_group_10th_df['math_score'].mean()
grade_group_11th_df = students_11th_df.groupby(['School'])
math_scores_11th_df = grade_group_11th_df['math_score'].mean()
grade_group_12th_df = students_12th_df.groupby(['School'])
math_scores_12th_df = grade_group_12th_df['math_score'].mean()

reading_scores_9th_df = grade_group_9th_df['reading_score'].mean()
reading_scores_10th_df = grade_group_10th_df['reading_score'].mean()
reading_scores_11th_df = grade_group_11th_df['reading_score'].mean()
reading_scores_12th_df = grade_group_12th_df['reading_score'].mean()


math_scores_by_grade = pd.DataFrame({'9th':math_scores_9th_df,"12th":math_scores_12th_df,'11th':math_scores_11th_df,
                                     '10th':math_scores_10th_df, })
math_scores_by_grade = math_scores_by_grade[['9th','10th','11th','12th']]
math_scores_by_grade

,9th,10th,11th,12th
School,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


# Reading Score by Grade

In [6]:
reading_scores_by_grade = pd.DataFrame({"12th":reading_scores_12th_df,'11th':reading_scores_11th_df,
                                     '10th':reading_scores_10th_df, '9th':reading_scores_9th_df})
reading_scores_by_grade = reading_scores_by_grade[['9th','10th','11th','12th']]
reading_scores_by_grade


,9th,10th,11th,12th
School,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


# Scores by School Spending

In [7]:
spend_bins = [0,585,615,645,675]
spend_labels = [" < $585","$585-615","$615-645","645 - 675"]
spend_categories = pd.cut(pass_count_merge_df["Per Student Budget"], spend_bins, labels=spend_labels)
breakdown_by_spending_df = pd.DataFrame({'Spend Level':spend_categories,
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
breakdown_by_spending_df.sort_values('Spend Level')
grouped_breakdown_by_spending_df = breakdown_by_spending_df.groupby(['Spend Level'])
spending_per_student = grouped_breakdown_by_spending_df.mean()
spending_per_student = spending_per_student[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                             '% Passing Reading', 'Overall Passing Rate']]
spending_per_student

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spend Level,,,,,
< $585,83.455,83.934,93.460,96.611,95.035
$585-615,83.600,83.885,94.231,95.900,95.066
$615-645,79.079,81.891,75.668,86.107,80.887
645 - 675,76.997,81.028,66.165,81.134,73.649


# Scores by School Size

In [8]:
school_size_bins =[0,1000,2000,5000]
school_size_labels = ['Small (< 1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
school_size_categories = pd.cut(pass_count_merge_df["size"], school_size_bins, labels=school_size_labels)
breakdown_by_size_df = pd.DataFrame({'School Size':school_size_categories,
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
grouped_breakdown_by_size_df = breakdown_by_size_df.groupby(['School Size'])
size_comparison = grouped_breakdown_by_size_df.mean()
size_comparison = size_comparison[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                   '% Passing Reading', 'Overall Passing Rate']]
size_comparison

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (< 1000),83.822,83.930,93.550,96.099,94.825
Medium (1000-2000),83.375,83.864,93.600,96.791,95.195
Large (2000-5000),77.746,81.344,69.963,82.767,76.365


# Scores by School Type
Compare results based on school type:

In [9]:
school_types = schools_df[['type','school_name']]
calcs_by_type_df = pd.DataFrame({'school_name':aves_merge_df['school_name'],
                                        'Average Math Score':aves_merge_df['math_score'],                                       
                                        'Average Reading Score':aves_merge_df['reading_score'],
                                        '% Passing Math':pass_count_merge_df['% Passing Math'], 
                                        '% Passing Reading':pass_count_merge_df['% Passing Reading'],
                                        'Overall Passing Rate':pass_count_merge_df['% Overall Passing']})
breakdown_by_type_df = pd.merge(calcs_by_type_df, school_types,on='school_name')
breakdown_by_type_df = breakdown_by_type_df.rename(index=str, columns={"type":"Type"})
grouped_breakdown_by_type_df = breakdown_by_type_df.groupby(['Type'])
chart_vs_dist_df = grouped_breakdown_by_type_df.mean()
chart_vs_dist_df = chart_vs_dist_df[['Average Math Score', 'Average Reading Score','% Passing Math',
                                     '% Passing Reading','Overall Passing Rate']]
chart_vs_dist_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,
Charter,83.474,83.896,93.621,96.586,95.104
District,76.957,80.967,66.548,80.799,73.674
